In [1]:
import sqlite3
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,mean_absolute_error
from math import sqrt
from datetime import datetime

In [2]:
cnx=sqlite3.connect('database.sqlite')
player_data=pd.read_sql_query("SELECT * FROM Player_Attributes",cnx)
player_data.head()

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,2015-09-21 00:00:00,62.0,66.0,right,medium,medium,49.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,4,218353,505942,2015-03-20 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,5,218353,505942,2007-02-22 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0


In [3]:
player_data.shape

(183978, 42)

In [4]:
player_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183978 entries, 0 to 183977
Data columns (total 42 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   id                   183978 non-null  int64  
 1   player_fifa_api_id   183978 non-null  int64  
 2   player_api_id        183978 non-null  int64  
 3   date                 183978 non-null  object 
 4   overall_rating       183142 non-null  float64
 5   potential            183142 non-null  float64
 6   preferred_foot       183142 non-null  object 
 7   attacking_work_rate  180748 non-null  object 
 8   defensive_work_rate  183142 non-null  object 
 9   crossing             183142 non-null  float64
 10  finishing            183142 non-null  float64
 11  heading_accuracy     183142 non-null  float64
 12  short_passing        183142 non-null  float64
 13  volleys              181265 non-null  float64
 14  dribbling            183142 non-null  float64
 15  curve            

In [5]:
numerical_player_data=[feature for feature in player_data.columns if player_data[feature].dtype!='O']
print(numerical_player_data)
categorical_player_data=[feature for feature in player_data.columns if player_data[feature].dtype=='O']
print(categorical_player_data)

['id', 'player_fifa_api_id', 'player_api_id', 'overall_rating', 'potential', 'crossing', 'finishing', 'heading_accuracy', 'short_passing', 'volleys', 'dribbling', 'curve', 'free_kick_accuracy', 'long_passing', 'ball_control', 'acceleration', 'sprint_speed', 'agility', 'reactions', 'balance', 'shot_power', 'jumping', 'stamina', 'strength', 'long_shots', 'aggression', 'interceptions', 'positioning', 'vision', 'penalties', 'marking', 'standing_tackle', 'sliding_tackle', 'gk_diving', 'gk_handling', 'gk_kicking', 'gk_positioning', 'gk_reflexes']
['date', 'preferred_foot', 'attacking_work_rate', 'defensive_work_rate']


In [6]:
player_data.isnull().sum()

id                        0
player_fifa_api_id        0
player_api_id             0
date                      0
overall_rating          836
potential               836
preferred_foot          836
attacking_work_rate    3230
defensive_work_rate     836
crossing                836
finishing               836
heading_accuracy        836
short_passing           836
volleys                2713
dribbling               836
curve                  2713
free_kick_accuracy      836
long_passing            836
ball_control            836
acceleration            836
sprint_speed            836
agility                2713
reactions               836
balance                2713
shot_power              836
jumping                2713
stamina                 836
strength                836
long_shots              836
aggression              836
interceptions           836
positioning             836
vision                 2713
penalties               836
marking                 836
standing_tackle     

In [7]:
player_data.dropna(thresh=5,inplace=True)

In [8]:
player_data.shape

(183142, 42)

In [9]:
target_data=player_data['overall_rating']
target_data

0         67.0
1         67.0
2         62.0
3         61.0
4         61.0
          ... 
183973    83.0
183974    78.0
183975    77.0
183976    78.0
183977    80.0
Name: overall_rating, Length: 183142, dtype: float64

In [10]:
from Clean_and_Merge import Clean_and_Merge
custom_attribute_object=Clean_and_Merge()

merged_player_data=custom_attribute_object.fit_transform(player_data)
merged_player_data

,potential,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,...,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,preferred_foot,attacking_work_rate,defensive_work_rate
0,67.600000,48.600000,43.600000,70.600000,60.600000,43.600000,50.600000,44.600000,38.600000,63.600000,...,66.000000,67.800000,5.600000,10.600000,9.600000,7.600000,7.600000,right,medium,medium
1,74.484848,70.787879,49.454545,52.939394,62.272727,29.151515,61.090909,61.878788,62.121212,63.242424,...,68.787879,71.515152,12.181818,8.666667,14.242424,10.363636,12.909091,left,medium,medium
2,74.192308,68.115385,57.923077,58.692308,65.115385,54.269231,69.038462,60.192308,55.615385,60.461538,...,21.115385,21.346154,14.038462,11.807692,17.730769,10.115385,13.500000,right,medium,medium
3,70.782609,57.217391,26.260870,69.260870,64.695652,47.782609,55.565217,37.782609,40.391304,60.826087,...,70.652174,68.043478,14.173913,11.173913,22.869565,11.173913,10.173913,right,medium,medium
4,74.680000,45.080000,38.840000,73.040000,64.760000,32.080000,50.600000,45.480000,26.360000,56.840000,...,76.040000,74.600000,8.280000,8.320000,24.920000,12.840000,11.920000,right,medium,medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11055,75.461538,42.000000,27.000000,75.153846,70.000000,30.000000,56.384615,49.000000,35.692308,54.538462,...,76.307692,72.692308,12.846154,12.769231,25.461538,11.384615,13.615385,right,low,medium
11056,71.625000,67.250000,46.750000,60.312500,63.125000,54.562500,61.187500,63.812500,61.750000,62.312500,...,65.250000,63.875000,9.000000,8.000000,5.000000,5.000000,8.000000,left,medium,medium
11057,72.857143,63.142857,44.571429,59.857143,66.857143,57.000000,66.857143,60.000000,52.571429,58.285714,...,64.285714,59.000000,7.142857,18.714286,42.428571,16.714286,16.714286,left,NaN,NaN
11058,78.125000,46.750000,43.000000,79.000000,58.875000,59.000000,36.000000,29.000000,41.375000,50.625000,...,73.250000,65.000000,14.000000,18.500000,41.625000,19.000000,20.000000,right,NaN,NaN


In [12]:
from Clean_Merge_Target import Clean_and_Merge_Target

merged_target_object=Clean_and_Merge_Target()
merged_target_data=merged_target_object.fit_transform(player_data)
merged_target_data

,overall_rating
0,63.600000
1,66.969697
2,67.000000
3,69.086957
4,73.240000
...,...
11055,74.384615
11056,65.687500
11057,67.571429
11058,70.750000


In [14]:
X_train,X_test,Y_train,Y_test=train_test_split(merged_player_data,
                                               merged_target_data,
                                               test_size=0.25,
                                               random_state=101)

In [15]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

In [40]:
num_pipeline=Pipeline([
    ('num_imputer',SimpleImputer(strategy='mean')),
    ('num_scaler',StandardScaler())
])
cat_pipeline=Pipeline([
    ('cat_imputer',SimpleImputer(strategy='most_frequent')),
    ('cat_encoder',OneHotEncoder(handle_unknown='ignore'))
])

full_transformer=ColumnTransformer([
    ('num_transfromer',num_pipeline,numerical_player_data),
    ('cat_transformer',cat_pipeline,categorical_player_data)
])

target_pipeline=Pipeline([
    ('num_imputer',SimpleImputer(strategy='mean',add_indicator=True)),
    ('num_scaler',StandardScaler())
])

In [41]:
full_transformed_data=full_transformer.fit_transform(X_train)
full_transformed_data

array([[ 0.10090255,  1.22755154,  0.67110896, ...,  0.        ,
         0.        ,  1.        ],
       [-1.08743191, -0.42570612,  0.95348586, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.97050581,  0.82206878,  0.61402092, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 0.61640936, -0.4951807 , -0.94495578, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.78001385, -0.15933825, -0.54854915, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.91902661,  0.97092842,  0.74322075, ...,  0.        ,
         1.        ,  0.        ]])

In [43]:
full_transformed_target=target_pipeline.fit_transform(pd.DataFrame(Y_train))
full_transformed_target

array([[ 0.50504521],
       [-0.47663795],
       [ 0.81982371],
       ...,
       [ 1.23550264],
       [ 0.31427609],
       [ 0.88549179]])

### This function is used to create the filename based upon the date and time

In [44]:
def generate_file_name(filename):
    now = datetime.now()
    current = now.strftime("%d_%m_%Y-%H_%M_%S")
    
    return now.strftime("%d_%m_%Y-%H_%M_%S") + '_' + filename 

### Importing libraries to save the trained pipelines upon the data

In [45]:
import joblib
import os

In [46]:
dir_name = 'pipelines'
if not os.path.isdir(dir_name):
    os.makedirs(dir_name)
    
    
joblib.dump(custom_attribute_object, os.path.join(dir_name, generate_file_name('custom_attribute_object.pkl')))
joblib.dump(merged_target_object, os.path.join(dir_name, generate_file_name('merged_target_object.pkl')))    
joblib.dump(full_transformer, os.path.join(dir_name, generate_file_name('full_transformer.pkl')))
joblib.dump(target_pipeline, os.path.join(dir_name, generate_file_name('target_pipeline.pkl')))

['pipelines\\30_08_2024-15_58_22_target_pipeline.pkl']

In [47]:
full_transformed_data.shape

(8295, 43)

In [48]:
full_transformed_target.shape

(8295, 1)

### Performing the learning process for multiple models

In [50]:
from sklearn.model_selection import cross_val_score

In [54]:
%%time
lin_reg=LinearRegression()

cross_val_score(estimator=lin_reg,X=full_transformed_data,
               y=full_transformed_target,cv=10,n_jobs=1)

CPU times: total: 391 ms
Wall time: 243 ms


array([0.8832075 , 0.85393926, 0.85278637, 0.85950952, 0.86948382,
       0.86839887, 0.87430382, 0.87062585, 0.86851204, 0.86427286])

In [55]:
%%time
tree_reg=DecisionTreeRegressor()
cross_val_score(estimator=tree_reg,X=full_transformed_data,
               y=full_transformed_target,cv=10,n_jobs=1)

CPU times: total: 2.22 s
Wall time: 2.27 s


array([0.87150736, 0.83273322, 0.84373402, 0.8729458 , 0.85466211,
       0.85579992, 0.87345417, 0.86378091, 0.87297423, 0.86097164])

In [56]:
%%time
from sklearn.svm import SVR
svm_reg=SVR()
cross_val_score(estimator=svm_reg,X=full_transformed_data,
               y=full_transformed_target,cv=10,n_jobs=1)

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,

CPU times: total: 16.1 s
Wall time: 17.1 s


array([0.97979895, 0.97371579, 0.97729247, 0.9732263 , 0.96967114,
       0.97856211, 0.97959338, 0.97480097, 0.97823914, 0.97159468])

In [57]:
%%time
from xgboost import XGBRegressor
xgb_reg=XGBRegressor()
cross_val_score(estimator=xgb_reg,X=full_transformed_data,
               y=full_transformed_target,cv=10,n_jobs=1)

CPU times: total: 24 s
Wall time: 2.21 s


array([0.95969705, 0.95508884, 0.95474635, 0.95063812, 0.95374513,
       0.95765224, 0.95863455, 0.95901207, 0.95907248, 0.96085102])

In [58]:
test_data=full_transformer.transform(X_test)
test_data

array([[ 1.01426748,  0.72102681,  1.49658359, ...,  0.        ,
         1.        ,  0.        ],
       [-1.21937101,  0.13257175,  0.28736598, ...,  0.        ,
         0.        ,  1.        ],
       [ 1.91764261,  1.19837496,  0.04261739, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 1.64737683,  1.16164806,  0.91932586, ...,  1.        ,
         0.        ,  0.        ],
       [-0.36388125, -0.12847485, -0.32555682, ...,  0.        ,
         0.        ,  1.        ],
       [ 1.5942889 ,  1.7453745 ,  1.12428583, ...,  0.        ,
         0.        ,  1.        ]])

In [59]:
best_model_details={}

### Computing the errors for all the model to find out the best model

In [69]:
lin_reg.fit(full_transformed_data,full_transformed_target)

y_pred=lin_reg.predict(test_data)
y_pred.reshape(-1,1)
test_target=target_pipeline.transform(pd.DataFrame(Y_test))

error=sqrt(mean_squared_error(test_target,y_pred))
print("Error :",error)
best_model_Details[lin_reg]=error

Error : 0.3614413323357659


In [70]:
tree_reg.fit(full_transformed_data,full_transformed_target)

y_pred=tree_reg.predict(test_data)
y_pred.reshape(-1,1)
test_target=target_pipeline.transform(pd.DataFrame(Y_test))

error=sqrt(mean_squared_error(test_target,y_pred))
print("Error :",error)
best_model_Details[tree_reg]=error

Error : 0.3581723445963566


In [72]:
svm_reg.fit(X=full_transformed_data,y=full_transformed_target)

y_pred=svm_reg.predict(test_data)
y_pred.reshape(-1,1)
test_target=target_pipeline.transform(pd.DataFrame(Y_test))

error=sqrt(mean_squared_error(test_target,y_pred))
print("Error :",error)
best_model_Details[svm_reg]=error

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Error : 0.1505948045124886


In [73]:
xgb_reg.fit(X=full_transformed_data,y=full_transformed_target)

y_pred=xgb_reg.predict(test_data)
y_pred.reshape(-1,1)
test_target=target_pipeline.transform(pd.DataFrame(Y_test))

error=sqrt(mean_squared_error(test_target,y_pred))
print("Error :",error)
best_model_Details[xgb_reg]=error

Error : 0.1983389446465487


In [74]:
best_model_Details

{LinearRegression(): 0.3614413323357659,
 DecisionTreeRegressor(): 0.3581723445963566,
 SVR(): 0.1505948045124886,
 XGBRegressor(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...): 0.1983389446465487}

In [75]:
min_value=list(best_model_Details.values())[0]
my_model=''

In [76]:
for key,value in best_model_Details.items():
    if value<min_value:
        min_value=value
        my_model=key
print(f"Best performing model: {my_model} with error: {error}")

Best performing model: SVR() with error: 0.1983389446465487


In [77]:
dir_name = 'models'
if not os.path.isdir(dir_name):
    os.makedirs(dir_name)
    
joblib.dump(my_model, os.path.join(dir_name, generate_file_name('best_model.pkl')))

['models\\30_08_2024-16_25_57_best_model.pkl']